<a 
href="https://colab.research.google.com/github/wingated/cs474_labs_f2019/blob/master/DL_Lab6.ipynb"
  target="_parent">
  <img
    src="https://colab.research.google.com/assets/colab-badge.svg"
    alt="Open In Colab"/>
</a>

# Lab 6: Sequence-to-sequence models

### Description:
For this lab, you will code up the [char-rnn model of Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). This is a recurrent neural network that is trained probabilistically on sequences of characters, and that can then be used to sample new sequences that are like the original.

This lab will help you develop several new skills, as well as understand some best practices needed for building large models. In addition, we'll be able to create networks that generate neat text!

### Deliverable:
- Fill in the code for the RNN (using PyTorch's built-in GRU).
- Fill in the training loop
- Fill in the evaluation loop. In this loop, rather than using a validation set, you will sample text from the RNN.
- Implement your own GRU cell.
- Train your RNN on a new domain of text (Star Wars, political speeches, etc. - have fun!)

### Grading Standards:
- 20% Implementation the RNN
- 20% Implementation training loop
- 20% Implementation of evaluation loop
- 20% Implementation of your own GRU cell
- 20% Training of your RNN on a domain of your choice

### Tips:
- Read through all the helper functions, run them, and make sure you understand what they are doing
- At each stage, ask yourself: What should the dimensions of this tensor be? Should its data type be float or int? (int is called `long` in PyTorch)
- Don't apply a softmax inside the RNN if you are using an nn.CrossEntropyLoss (this module already applies a softmax to its input).

### Example Output:
An example of my final samples are shown below (more detail in the
final section of this writeup), after 150 passes through the data.
Please generate about 15 samples for each dataset.

<code>
And ifte thin forgision forward thene over up to a fear not your
And freitions, which is great God. Behold these are the loss sub
And ache with the Lord hath bloes, which was done to the holy Gr
And appeicis arm vinimonahites strong in name, to doth piseling 
And miniquithers these words, he commanded order not; neither sa
And min for many would happine even to the earth, to said unto m
And mie first be traditions? Behold, you, because it was a sound
And from tike ended the Lamanites had administered, and I say bi
</code>


---

## Part 0: Readings, data loading, and high level training

---

There is a tutorial here that will help build out scaffolding code, and get an understanding of using sequences in pytorch.

* Read the following

> * [Pytorch sequence-to-sequence tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) (You will be implementing the decoder, not the encoder, as we are not doing sequence-to-sequence translation.)
* [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)






In [2]:
! wget -O ./text_files.tar.gz 'https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz' 
! tar -xzf text_files.tar.gz
! pip install unidecode
! pip install torch

import unidecode
import string
import random
import re
 
import pdb
 
all_characters = string.printable
n_characters = len(all_characters)
file = unidecode.unidecode(open('./text_files/lotr.txt').read())
file_len = len(file)
print('file_len =', file_len)

--2022-10-16 03:22:38--  https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz
Resolving piazza.com (piazza.com)... 18.215.228.240, 52.200.164.228, 18.207.91.3, ...
Connecting to piazza.com (piazza.com)|18.215.228.240|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-uploads.piazza.com/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz [following]
--2022-10-16 03:22:39--  https://cdn-uploads.piazza.com/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz
Resolving cdn-uploads.piazza.com (cdn-uploads.piazza.com)... 13.226.228.49, 13.226.228.46, 13.226.228.50, ...
Connecting to cdn-uploads.piazza.com (cdn-uploads.piazza.com)|13.226.228.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1533290 (1.5M) [application/x-gzip]
Saving to: ‘./text_files.tar.gz’

./text_files.tar.gz 100%[===================>]   1.46M  2.04MB/

In [3]:
chunk_len = 200
 
def random_chunk():
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]
  
print(random_chunk())

 great wizard than Gandalf did, 
more kingly, beautiful, and powerful; and older. Yet by a sense other than 
sight Pippin perceived that Gandalf had the greater power and the deeper 
wisdom, and a maje


In [4]:
import torch
# Turn string into list of longs
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
      tensor[c] = all_characters.index(string[c])
  return tensor

print(char_tensor('abcDEF'))

tensor([10, 11, 12, 39, 40, 41])


---

## Part 4: Creating your own GRU cell 

**(Come back to this later - its defined here so that the GRU will be defined before it is used)**

---

The cell that you used in Part 1 was a pre-defined Pytorch layer. Now, write your own GRU class using the same parameters as the built-in Pytorch class does.

Please do not look at the documentation's code for the GRU cell definition. The answer is right there in the code, and in theory, you could just cut-and-paste it. This bit is on your honor!

**TODO:**
* Create a custom GRU cell

**DONE:**



In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class GRU(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
    super(GRU, self).__init__()
    self.n_layers = num_layers
    self.hz = []
    self.inn = []
    self.hn = []
    self.ir = []
    self.hr = []
    self.iz = []
    
    for i in range(self.n_layers):
      self.hz.append(nn.Linear(hidden_size, hidden_size))
      self.inn.append(nn.Linear(input_size,  hidden_size))
      self.hn.append(nn.Linear(hidden_size, hidden_size))
      self.ir.append(nn.Linear(input_size,  hidden_size))
      self.hr.append(nn.Linear(hidden_size, hidden_size))
      self.iz.append(nn.Linear(input_size,  hidden_size))
      
    
  
  def forward(self, inputs, hidden):
    # Each layer does the following:
    # r_t = sigmoid(W_ir*x_t + b_ir + W_hr*h_(t-1) + b_hr)
    # z_t = sigmoid(W_iz*x_t + b_iz + W_hz*h_(t-1) + b_hz)
    # n_t = tanh(W_in*x_t + b_in + r_t**(W_hn*h_(t-1) + b_hn))
    # h_(t) = (1 - z_t)**n_t + z_t**h_(t-1)
    # Where ** is hadamard product (not matrix multiplication, but elementwise multiplication)
    for i in range(self.n_layers):
      r_t = nn.Sigmoid(self.ir[i](inputs) + self.hr[i](hidden))
      z_t = nn.Sigmoid(self.iz[i](inputs) + self.hz[i](hidden))
      n_t = nn.Tanh(self.inn[i](inputs) + r_t*self.hn[i](hidden))
      hidden = (1 - z_t) * n_t + z_t * hidden
      output = hidden[-1:]

    
    return output, hidden
  


---

##  Part 1: Building a sequence to sequence model

---

Great! We have the data in a useable form. We can switch out which text file we are reading from, and trying to simulate.

We now want to build out an RNN model, in this section, we will use all built in Pytorch pieces when building our RNN class.


**TODO:**
* Create an RNN class that extends from nn.Module.

**DONE:**



In [16]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, n_layers=1):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.n_layers = n_layers
    self.embedding = nn.Embedding(input_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size, num_layers=self.n_layers)
    self.lin = nn.Linear(hidden_size, output_size)

  def forward(self, input_char, hidden):
    # by reviewing the documentation, construct a forward function that properly uses the output
    # of the GRU
    output = self.embedding(input_char).view(1, 1, -1)
    output, hidden = self.gru(output, hidden)
    output = F.relu(output)
    output = self.lin(output)
    return output, hidden

  def init_hidden(self):
    return torch.zeros(self.n_layers, 1, self.hidden_size)

In [13]:
def random_training_set():    
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target

---

## Part 2: Sample text and Training information

---

We now want to be able to train our network, and sample text after training.

This function outlines how training a sequence style network goes. 

**TODO:**
* Fill in the pieces.

**DONE:**




In [21]:
# NOTE: decoder_optimizer, decoder, and criterion will be defined below as global variables
def train(input, target):
  ## initialize hidden layers, set up gradient and loss 
    # your code here
  ## /
  decoder_optimizer.zero_grad()
  hidden = decoder.init_hidden()
  loss = 0
  for i, tar in zip(input, target):
    char, hidden = decoder(i, hidden)
    tar = tar.unsqueeze(0)
    loss += criterion(char.squeeze(0), tar)
  loss.backward()
  decoder_optimizer.step()
  return loss.item()/len(input)
    

---

## Part 3: Sample text and Training information

---

You can at this time, if you choose, also write out your train loop boilerplate that samples random sequences and trains your RNN. This will be helpful to have working before writing your own GRU class.

If you are finished training, or during training, and you want to sample from the network you may consider using the following function. If your RNN model is instantiated as `decoder`then this will probabilistically sample a sequence of length `predict_len`

**TODO:**
* Fill out the evaluate function to generate text frome a primed string

**DONE:**



In [17]:
def sample_outputs(output, temperature):
    """Takes in a vector of unnormalized probability weights and samples a character from the distribution"""
    # As temperature approaches 0, this sampling function becomes argmax (no randomness)
    # As temperature approaches infinity, this sampling function becomes a purely random choice
    return torch.multinomial(torch.exp(output / temperature), 1)

def evaluate(prime_str='A', predict_len=100, temperature=0.8):
  ## initialize hidden state, initialize other useful variables
    # your code here
  ## /
  hidden = decoder.init_hidden()
  prediction = prime_str + ''
  primer = char_tensor(prime_str) 
  for char in primer[:-1]:
    x, hidden = decoder(char, hidden)
  last_char = primer[-1]
  
  for i in range(predict_len):
    pred, hidden = decoder(last_char, hidden)
    index = sample_outputs(pred.squeeze(0), temperature)
    decoded = all_characters[index]
    last_char = char_tensor(decoded)
    prediction += decoded
      
  return prediction

---

## Part 4: (Create a GRU cell, requirements above)

---



---

## Part 5: Run it and generate some text!

---


**TODO:** 
* Create some cool output

**DONE:**




Assuming everything has gone well, you should be able to run the main function in the scaffold code, using either your custom GRU cell or the built in layer, and see output something like this. I trained on the “lotr.txt” dataset, using chunk_length=200, hidden_size=100 for 2000 epochs. These are the results, along with the prime string:

---

 G:
 
 Gandalf was decrond. 
'All have lord you. Forward the road at least walk this is stuff, and 
went to the long grey housel-winding and kindled side was a sleep pleasuring, I do long 
row hrough. In  

 lo:
 
 lost death it. 
'The last of the gatherings and take you,' said Aragorn, shining out of the Gate. 
'Yes, as you there were remembaused to seen their pass, when? What 
said here, such seven an the sear 

 lo:
 
 low, and frod to keepn 
Came of their most. But here priced doubtless to an Sam up is 
masters; he left hor as they are looked. And he could now the long to stout in the right fro horseless of 
the like 

 I:
 
 I had been the 
in his eyes with the perushed to lest, if then only the ring and the legended 
of the less of the long they which as the 
enders of Orcovered and smood, and the p 

 I:
 
 I they were not the lord of the hoomes. 
Home already well from the Elves. And he sat strength, and we 
housed out of the good of the days to the mountains from his perith. 

'Yess! Where though as if  

 Th:
 
 There yarden 
you would guard the hoor might. Far and then may was 
croties, too began to see the drumbred many line 
and was then hoard walk and they heart, and the chair of the 
Ents of way, might was 

 G:
 
 Gandalf 
been lat of less the round of the stump; both and seemed to the trees and perished they 
lay are speered the less; and the wind the steep and have to she 
precious. There was in the oonly went 

 wh:
 
 which went out of the door. 
Hull the King and of the The days of his brodo 
stumbler of the windard was a thing there, then it been shining langing 
to him poor land. They hands; though they seemed ou 

 ra:
 
 rather,' have all the least deather 
down of the truven beginning to the house of sunk. 
'Nark shorts of the Eyes of the Gate your great nothing as Eret. 
'I wander trust horn, and there were not, it  

 I:
 
 I can have no mind 
together! Where don't may had one may little blung 
terrible to tales. And turn and Gandalf shall be not to as only the Cattring 
not stopped great the out them forms. On they she lo 

---


In [18]:
import time
n_epochs = 2000
print_every = 200
plot_every = 10
hidden_size = 200
n_layers = 3
lr = 0.001
 
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
start = time.time()
all_losses = []
loss_avg = 0

In [22]:
# n_epochs = 2000
for epoch in range(1, n_epochs + 1):
  loss_ = train(*random_training_set())       
  loss_avg += loss_

  if epoch % print_every == 0:
      print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
      print(evaluate('Wh', 100), '\n')

  if epoch % plot_every == 0:
      all_losses.append(loss_avg / plot_every)
      loss_avg = 0

[190.60812306404114 (200 10%) 2.8546]
Wh e d mhml htt seeehr iH le  hdld, ittadiae nn aftinafmneh ,adnt eeoiiaee eih a y, uhr s' smasa e n n 

[321.4662744998932 (400 20%) 2.6009]
Wh  arnirt nome -afrdeur atln noh nwie tdolle oboee eet es waor Rbsa oe noe 
aimee ond nod hue ah rh.e 

[450.3840799331665 (600 30%) 2.5767]
Whe herge ceoivesdhen hte anuel on she covy Enn fosmoth. undAs ent whroas to faos. uore weoath he ders 

[578.66535115242 (800 40%) 2.1709]
Whis and the penger stould the an free. sind drorred fow ward dowttet tento Fhver rind had wearn 
teri 

[709.0300748348236 (1000 50%) 1.9727]
Whing, said 

Gandring bado. 'But but Sam the Hips as cardogold and hidden paleding beth taden, and th 

[838.7990665435791 (1200 60%) 1.8089]
Where 
leat it,s eyes. It a light of the metun from lieaden stailed 
it yelike to air. I tried as and  

[967.4614770412445 (1400 70%) 1.7571]
Wh? 'He hime se-gent wordden it the 
wicomks hinlis,' sho a seel warkly be as we he clime and passey,  

[109

In [23]:
for i in range(10):
  start_strings = [" Th", " wh", " he", " I ", " ca", " G", " lo", " ra"]
  start = random.randint(0,len(start_strings)-1)
  print(start_strings[start])
#   all_characters.index(string[c])
  print(evaluate(start_strings[start], 200), '\n')

 I 
 I had road. He had now that which I silent some, now he hacmed, and it 
far be 
all some them leading it wish when you Andol, the arvall the door it them. 

'I walking of there weth yet 
For the stood e 

 lo
 long for 
many from betward him Load it the greep the 
shall host his south the chout water 
lealling could there goed in thh ham the bagn and a dapder do flecied deep of the last and the hoads at befor 

 G
 Gines men no paliping to hiys and 
sursed the most of the shagotted midder betwawn his back the passed for will sterd. The lam, and be should coning Scame wonder the howly that you pers which that meen 

 lo
 long tom the right that stronger. 'He pook we arts of southy never uping who stided in the oft. Black then a was hall,' said Light to the felt on the for 
atted it icling 
before come the for the words  

 G
 Gandalf with story your splak at Gafly. Though cra one to the gloor of he have lay, and that the house him. Frodo have have sorched time uncil and they rouve 

---

## Part 6: Generate output on a different dataset

---

**TODO:**

* Choose a textual dataset. Here are some [text datasets](https://www.kaggle.com/datasets?tags=14104-text+data%2C13205-text+mining) from Kaggle 

* Generate some decent looking results and evaluate your model's performance (say what it did well / not so well)

**DONE:**



In [ ]:
file = unidecode.unidecode(open('./text_files/tiny_shakespeare.txt').read())
file_len = len(file)
print('file_len =', file_len)

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(n_epochs):
  loss_ = train(*random_training_set())       
  loss_avg += loss_

  if epoch % print_every == 0:
    print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
    print(evaluate('Wh', predict_len=100, temperature=0.6), '\n')

  if epoch % plot_every == 0:
    all_losses.append(loss_avg / plot_every)
    loss_avg = 0

In [ ]:
for i in range(10):
  start_strings = [" Th", " wh", " he", " I ", " ca", " G", " lo", " ra"]
  start = random.randint(0,len(start_strings)-1)
  print(start_strings[start])
#   all_characters.index(string[c])
  print(evaluate(start_strings[start], 200), '\n')

 ca
 can suce conto.

LADWICS:
I, shall is gailost his be coming thew mistain,
As this and Poss, and sees love in long intis turn'd to
hould the blood could be and there of that fear in thef
the say he gook  

 he
 her gods hrow.
Have thee, and then the grace.

BIBTASTAR:
Yon? we with in my see, wing gone bake, come,
That this a pits with one peass.

DUKEU:
Vaintite dober more love a colcoke her a croped,

BUCCILA 

 wh
 whom you had then we
God, by but me ire forst, this being nend,
Whither with my spaw in sown wair you hadst your kindre us must not some,
By sceeus the roon.

FRILGO ENIO:
I pass so her! wonk, Tranaty a 

 I 
 I and the must they being me.

LUCIO:
My live!

LEONDA:
But that Gurst, ristess werion and my had?

MENENANSUONE:
Nor, in hand mady the king will dead you real'd,
And this forful contery thou form'd wer 

 Th
 Thath met all well,
A modannand all in protor and that state; this indein, be gone:
Most and me am on that hold of he ruse tomebher appus.

Stree:
Where yo

Positives:

*   It generally was able to generate full words
*   A lot of the text actually sounds Shakespearean

Negatives:


*  The sentences are almost always unintelligible( then again so are most shakespearean texts)
*  It throws new lines in often when they shouldn't be there

